In [1]:
import pandas as pd

import sys
sys.path.append("src")

from merging_with_id import merging, create_id

In [20]:
# load datasets
datasets = {}
years = range(2019, 2021)
for year in years:
    print(f"Schools stats {year} Loading...")
    file_name = f'ZZSO-{year}.xlsx'
    dataset = pd.read_excel(f"./fin_data/{file_name}", header=1, dtype='str')
    datasets.update({year:dataset})
    print("success")

Schools stats 2019 Loading...
success
Schools stats 2020 Loading...
success


In [21]:
for year in years:
    print(year,'\n', list(datasets[year].columns))

2019 
 ['№ Області', 'Область', 'Тип школи: міська / сільська', '№№', 'Назва тер. Одиниці', 'Найменування закладу', 'Ступінь закладу ( I, I-II, I-III)', 'Загальна площа приміщень школи ( без господарських будівель) (кв.м.)', 'Кількість штатних працівників педагогічного персоналу  разом', 'Кількість штатних працівників непедагогічного персоналу  разом', 'Кількість штатних працівників педагогічного персоналу  пенсійного віку', 'Питома вага непедагогічного персоналу у загальній кількості працівників, %**', 'Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %**', 'Кількість учнів разом', 'Разом кількість учнів, та дітей, які здобувають дошкільну освіту', 'Кількість класів разом', 'Фактична наповнюваність класів, учнів на клас**', 'Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя**', 'Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (касові видатки)', 'Видатки на функціонування закладу (загал

In [22]:
rename_dict = {
    "Область": "regname",
    "Тип територіальної одиниці": "tertype",
    "Тип школи: міська / сільська": 'eotype',
    "Назва тер. Одиниці": "areaname",
    "Найменування закладу": "eoname",
    "Ступінь закладу ( I, I-II, I-III)": "eolevel",
    "Загальна площа приміщень школи ( без господарських будівель) (кв.м.)": "eosize",
    "Кількість штатних працівників педагогічного персоналу  разом": "teachstuff",
    "Кількість штатних працівників непедагогічного персоналу  разом": "nonteachstuff",
    "Кількість штатних працівників педагогічного персоналу  пенсійного віку": "teachstuffretage",
    "Кількість учнів разом": "pupils",
    "Кількість класів разом": "classes",
    "Разом кількість учнів, та дітей, які здобувають дошкільну освіту": "pupilswithpreschool",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2018рік (касові видатки)": "opex",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (касові видатки)": "opex",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (касові видатки)": "opex",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (план)": "opexplan",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (план)": "opexplan",
    "Видатки на функціонування закладу (загальний фонд), тис.грн.\n2021 рік (план)": "opexplan",
    "Опорна": "hub",
    "Код бюджету": "budgetcode",
    "Форма власності": "ownership",
    "Код ДІСО": "DISO",
    "ЄДРПОУ": "EDRPOU"
    }

In [24]:
for year in years:
    datasets[year].rename(columns=rename_dict, inplace=True)
    to_drop = [col for col in datasets[year].columns if col not in rename_dict.values()]
    datasets[year].drop(columns = to_drop, inplace = True)
    datasets[year]['year'] = year

school_finance = pd.concat([ datasets[2019], datasets[2020]], ignore_index=True, sort=False)
school_finance.columns


Index(['regname', 'eotype', 'areaname', 'eoname', 'eolevel', 'eosize',
       'teachstuff', 'nonteachstuff', 'teachstuffretage', 'pupils',
       'pupilswithpreschool', 'classes', 'opex', 'opexplan', 'hub',
       'budgetcode', 'ownership', 'DISO', 'EDRPOU', 'year'],
      dtype='object')

## Location region

In [25]:
# load location dictionary
locations = pd.read_csv('./final_tables/locations.csv', encoding='utf-8', dtype=str)
locations

,regname,areaname,tername,KOATUU_2020,KATOTTG_2023
0,Запорізька область,Мелітопольський район,с.Терпіння,2323085101,UA23080270010078454
1,Хмельницька область,Красилівський район,м.Красилів,6822710100,UA68040210010032567
2,Дніпропетровська область,Петропавлівський район,с.Дмитрівка,1223881501,UA12140170040016918
3,Чернівецька область,м.Чернівці,Шевченківський район міста,7310100000,UA73060610010033137
4,Миколаївська область,Врадіївський район,с.Кумарі,4822383001,UA48080050190079797
...,...,...,...,...,...
29214,США,м.Детройт,м.Детройт,0026010000,OC26010000000000000
29215,Велика Британія,м.Абериствіт,м.Абериствіт,0008030000,OC08030000000000000
29216,США,м.Нью-Йорк,м.Нью-Йорк,0026020000,OC26020000000000000
29217,США,м.Х'юстон,м.Х'юстон,0026040000,OC26040000000000000


In [26]:
school_finance.regname = school_finance.regname.apply(lambda x: 'м.Київ' if x == 'місто Київ' or x == 'м.Київ' else x+' область')
school_finance.regname.unique()

array(['Вінницька область', 'Волинська область',
       'Дніпропетровська область', 'Донецька область',
       'Житомирська область', 'Закарпатська область',
       'Запорізька область', 'Івано-Франківська область',
       'Київська область', 'Кіровоградська область', 'Луганська область',
       'Львівська область', 'Миколаївська область', 'Одеська область',
       'Полтавська область', 'Рівненська область', 'Сумська область',
       'Тернопільська область', 'Харківська область',
       'Херсонська область', 'Хмельницька область', 'Черкаська область',
       'Чернівецька область', 'Чернігівська область', 'м.Київ'],
      dtype=object)

In [27]:
locations['KOATUU_2020_reg'] = locations['KOATUU_2020'].str[:2]
locations

,regname,areaname,tername,KOATUU_2020,KATOTTG_2023,KOATUU_2020_reg
0,Запорізька область,Мелітопольський район,с.Терпіння,2323085101,UA23080270010078454,23
1,Хмельницька область,Красилівський район,м.Красилів,6822710100,UA68040210010032567,68
2,Дніпропетровська область,Петропавлівський район,с.Дмитрівка,1223881501,UA12140170040016918,12
3,Чернівецька область,м.Чернівці,Шевченківський район міста,7310100000,UA73060610010033137,73
4,Миколаївська область,Врадіївський район,с.Кумарі,4822383001,UA48080050190079797,48
...,...,...,...,...,...,...
29214,США,м.Детройт,м.Детройт,0026010000,OC26010000000000000,00
29215,Велика Британія,м.Абериствіт,м.Абериствіт,0008030000,OC08030000000000000,00
29216,США,м.Нью-Йорк,м.Нью-Йорк,0026020000,OC26020000000000000,00
29217,США,м.Х'юстон,м.Х'юстон,0026040000,OC26040000000000000,00


In [28]:
loc = locations[['regname', 'KOATUU_2020_reg']]
loc.drop_duplicates(inplace= True)
loc.reset_index(drop=True, inplace=True)
loc

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_23158/3378581834.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc.drop_duplicates(inplace= True)


,regname,KOATUU_2020_reg
0,Запорізька область,23
1,Хмельницька область,68
2,Дніпропетровська область,12
3,Чернівецька область,73
4,Миколаївська область,48
5,Донецька область,14
6,Тернопільська область,61
7,Кіровоградська область,35
8,Одеська область,51
9,Львівська область,46


In [29]:
school_finance = school_finance.merge(loc[['regname', 'KOATUU_2020_reg']], on=['regname'], how='left')
school_finance.head()

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,classes,opex,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg
0,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,8,539,...,20,9915.2,11123,ні,2202100000,комунальна,8173,26244113,2019,05
1,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,8,840,...,30,13692.2,13758.8,ні,2202100000,комунальна,8174,26244107,2019,05
2,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,11,981,...,36,16184.7,17254.2,ні,2202100000,комунальна,8175,26244136,2019,05
3,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,6,854,...,32,14321.9,14598.6,ні,2202100000,комунальна,8176,26244099,2019,05
4,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,8,864,...,33,15061,16011.5,ні,2202100000,комунальна,8177,23064971,2019,05


In [30]:
set(school_finance.eolevel.unique())

{' I-III ',
 '0',
 'I',
 'I-II',
 'I-III',
 'I-IIІ',
 'II',
 'II-III',
 'III',
 'ІІ-ІІІ'}

In [31]:
values_to_drop = ['I', 'I-II', 'II']

school_finance = school_finance[~school_finance.eolevel.isin(values_to_drop)]
school_finance.reset_index(drop=True, inplace=True)
school_finance

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,classes,opex,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg
0,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,8,539,...,20,9915.2,11123,ні,2202100000,комунальна,8173,26244113,2019,05
1,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,8,840,...,30,13692.2,13758.8,ні,2202100000,комунальна,8174,26244107,2019,05
2,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,11,981,...,36,16184.7,17254.2,ні,2202100000,комунальна,8175,26244136,2019,05
3,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,6,854,...,32,14321.9,14598.6,ні,2202100000,комунальна,8176,26244099,2019,05
4,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,8,864,...,33,15061,16011.5,ні,2202100000,комунальна,8177,23064971,2019,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18895,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,30,19606.3,25731.415,NaN,26000000000,комунальна,22883,23392178,2020,80
18896,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,34,22065.4,26469.2,NaN,26000000000,комунальна,23029,23392149,2020,80
18897,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,18,15793.70295055802,10294.196,NaN,26000000000,комунальна,23280,26187881,2020,80
18898,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,30,20353.2,13412.1,NaN,26000000000,комунальна,23154,22878447,2020,80


In [33]:
# check whether there are duplicates by name and location
school_finance[school_finance.duplicated(subset=['eoname', 'year', 'KOATUU_2020_reg', 'EDRPOU'], keep=False)]

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,classes,opex,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg


In [34]:
school_finance[school_finance.EDRPOU.str.strip().str.len() != 8].EDRPOU.unique()

array(['2141288', '2141360', '2141377', '2141408', '2141383', '2141710',
       '6669359', '6669419', '6669655', '6669738', '6669744', '6669750',
       '6669773', '6669786', '6669827', '6669833', '6669856', '6670121',
       '6669879', '6670606', '6670919', '6670664', '6670635', '6670753',
       '6671110', '6671149', '6671468', '6671497', '6671155', '6671422',
       '6671445', '6671267', '6671349', '6671178', '6671474', '6670368',
       '6670380', '6670405', '6670411', '6670428', '6670474', '6670374',
       '6670397', '6669313', '6669263', '6669721', '2142997', '6669454',
       '6669336', '6669253', '6669247', '6669394', '6671439', '6671451',
       '6671161', '6669804', '6669862', '6669649', '6669810', '6669796',
       '6669603', '6669840', '6670687', '6670693', '6670701', '6670776',
       '6670598', '6670670', '6670718', '6670569', '6670552', '6670575',
       '6670581', '6670730', '6670782', '6671480', '2143235', '2143301',
       '2143465', '2136382', '4054435', '2143867', 

In [35]:
school_finance.EDRPOU = school_finance.EDRPOU.apply(lambda x: x.zfill(8) if isinstance(x, str) and len(x) >= 6 else None)

In [37]:
school_finance[(school_finance.EDRPOU.isna())].eoname.unique()

array(['Турбівська загальноосвітня школа І-ІІІ ст. №2 _x000D_\nЛиповецької районної ради _x000D_\nВінницької області_x000D_\n_x000D_\n',
       'Середня загальноосвітня школа І - ІІІ ступенів села Уладівка Літинський район Вінницька область',
       'Середня загальноосвітня школа І-ІІІ ступенів с. Івча',
       'Комунальний заклад "Загальноосвітня школа І-ІІІ ступенів № 19 Вінницької міської ради"',
       'Старобільська загальноосвітня школа ІІ-ІІІ ступенів №4 Старобільської районної ради Луганської області'],
      dtype=object)

In [38]:
set_edrpou = {
    'Турбівська загальноосвітня школа І-ІІІ ст. №2 _x000D_\nЛиповецької районної ради _x000D_\nВінницької області_x000D_\n_x000D_\n': '21723820',
    'Середня загальноосвітня школа І - ІІІ ступенів села Уладівка Літинський район Вінницька область':'26229088',
    'Середня загальноосвітня школа І-ІІІ ступенів с. Івча':'26223660',
    'Комунальний заклад "Загальноосвітня школа І-ІІІ ступенів № 19 Вінницької міської ради"':'22103539',
    'Старобільська загальноосвітня школа ІІ-ІІІ ступенів №4 Старобільської районної ради Луганської області':'33199525'}

In [39]:
for school in set_edrpou:
    school_finance.loc[school_finance['eoname'] == school, 'EDRPOU'] = set_edrpou[school]

In [40]:
create_id(school_finance, 'eoname', 'id')

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opex,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,id
0,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,8,539,...,9915.2,11123,ні,2202100000,комунальна,8173,26244113,2019,05,загальноосвтняшкола2ступ1комунальнавласностмжм...
1,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,8,840,...,13692.2,13758.8,ні,2202100000,комунальна,8174,26244107,2019,05,навчальновиховнийкомплексзагальноосвтняшкола2с...
2,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,11,981,...,16184.7,17254.2,ні,2202100000,комунальна,8175,26244136,2019,05,загальноосвтняшкола2ступ3комунальнавласностмжм...
3,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,6,854,...,14321.9,14598.6,ні,2202100000,комунальна,8176,26244099,2019,05,загальноосвтняшкола2ступ4комунальнавластностте...
4,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,8,864,...,15061,16011.5,ні,2202100000,комунальна,8177,23064971,2019,05,навчальновиховнийкомплексзагальноосвтняшкола2с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18895,м.Київ,місто,місто Київ,гімназія № 237,I-III,NaN,52,23,2,905,...,19606.3,25731.415,NaN,26000000000,комунальна,22883,23392178,2020,80,гмназя237
18896,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,22065.4,26469.2,NaN,26000000000,комунальна,23029,23392149,2020,80,середнязагальноосвтняшкола2ступ195менвкудряшов...
18897,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,15793.70295055802,10294.196,NaN,26000000000,комунальна,23280,26187881,2020,80,спецалзованашкола2ступ73зпоглибленимвивченямук...
18898,м.Київ,місто,місто Київ,Спеціалізована школа № 131 Святошинського райо...,I-III,NaN,50,30,1,861,...,20353.2,13412.1,NaN,26000000000,комунальна,23154,22878447,2020,80,спецалзованашкола131мкив


# Preparation to merge with schools_edrpou dataset

# Merge with schools

In [42]:
schools_edrpou = pd.read_csv('./final_tables/schools_edrpou.csv', dtype=str)
schools_edrpou

,eoname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU,to_change
0,"Вiдокремлений структурний пiдроздiл ""Краматорс...",0510100000,05,[2023],краматорськийколеджпромисловостнформацйнихтехн...,04601943,NaN
1,"ВСП ""Технологічно-промисловий фаховий коледж В...",0510100000,05,[2022],технологчнопромисловийколеджвницькогонацональн...,00419667,NaN
2,Вище професійне училище №11 м. Вінниці,0510100000,05,"[2018, 2019, 2020, 2021, 2022, 2023]",вищепрофесйнеучилище11мвниц,03065891,NaN
3,Вище художнє професійно-технічне училище № 5 м...,0510100000,05,[2023],вищехудожнпрофесйнотехнчнеучилище5мвниц,02539890,NaN
4,Вище художнє професійно-технічне училище №5 м....,0510100000,05,"[2018, 2019, 2020, 2021, 2022]",вищехудожнпрофесйнотехнчнеучилище5мвниц,02539890,NaN
...,...,...,...,...,...,...,...
32052,Школа №25 І-ІІІ ступенів Шевченківського район...,8039100000,80,"[2016, 2017, 2018, 2019, 2020, 2021]",школа253ступмкив,22880786,NaN
32053,Школа №27 І-ІІІ ступенів Шевченківського район...,8039100000,80,[2016],школа273ступмкив,26125710,NaN
32054,Школа №70 І-ІІІ ступенів Шевченківського район...,8039100000,80,[2016],школа703ступмкив,22881828,NaN
32055,Школа №95 І-ІІІ ступенів Шевченківського район...,8039100000,80,[2017],школа953ступмкив,26125905,NaN


In [43]:
school_finance_edrpou = school_finance.merge(schools_edrpou[['id', 'KOATUU_2020_reg', 'EDRPOU']], on=['id', 'KOATUU_2020_reg'], how='left')

In [47]:
school_finance_edrpou[(school_finance_edrpou['EDRPOU_x'] != school_finance_edrpou['EDRPOU_y'])&school_finance_edrpou['EDRPOU_y'].notna()]

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU_x,year,KOATUU_2020_reg,id,EDRPOU_y


In [50]:
school_finance_edrpou.loc[school_finance_edrpou['EDRPOU_y'].notna(), 'EDRPOU_new'] = school_finance_edrpou.loc[school_finance_edrpou['EDRPOU_y'].notna(), 'EDRPOU_y']

In [52]:
school_finance_edrpou = school_finance_edrpou.drop(columns=['EDRPOU_y'], axis=1)
school_finance_edrpou = school_finance_edrpou.rename(columns = {'EDRPOU_x':'EDRPOU'})

In [54]:
school_finance_edrpou[school_finance_edrpou['EDRPOU_new'].isna()]

,regname,eotype,areaname,eoname,eolevel,eosize,teachstuff,nonteachstuff,teachstuffretage,pupils,...,opexplan,hub,budgetcode,ownership,DISO,EDRPOU,year,KOATUU_2020_reg,id,EDRPOU_new
0,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,8,539,...,11123,ні,2202100000,комунальна,8173,26244113,2019,05,загальноосвтняшкола2ступ1комунальнавласностмжм...,NaN
1,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,8,840,...,13758.8,ні,2202100000,комунальна,8174,26244107,2019,05,навчальновиховнийкомплексзагальноосвтняшкола2с...,NaN
2,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,11,981,...,17254.2,ні,2202100000,комунальна,8175,26244136,2019,05,загальноосвтняшкола2ступ3комунальнавласностмжм...,NaN
3,Вінницька область,місто,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,6,854,...,14598.6,ні,2202100000,комунальна,8176,26244099,2019,05,загальноосвтняшкола2ступ4комунальнавластностте...,NaN
4,Вінницька область,місто,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,8,864,...,16011.5,ні,2202100000,комунальна,8177,23064971,2019,05,навчальновиховнийкомплексзагальноосвтняшкола2с...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20254,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 23 з пог...,I-III,NaN,49,26,2,723,...,31694.8,NaN,26000000000,комунальна,22929,03874432,2020,80,спецалзованашкола2ступ23зпоглибленимвивченяман...,NaN
20255,м.Київ,місто,місто Київ,Школа І-ІІІ ступенів № 151 міста Києва,I-III,NaN,26,23,1,475,...,17399.100000000002,NaN,26000000000,комунальна,22851,23392132,2020,80,школа2ступ151мкив,NaN
20257,м.Київ,місто,місто Київ,Середня загальноосвітня школа І-ІІІ ступенів №...,I-III,NaN,58,26,2,827,...,26469.2,NaN,26000000000,комунальна,23029,23392149,2020,80,середнязагальноосвтняшкола2ступ195менвкудряшов...,NaN
20258,м.Київ,місто,місто Київ,Спеціалізована школа І-ІІІ ступенів № 73 з пог...,I-III,NaN,32,22,1,483,...,10294.196,NaN,26000000000,комунальна,23280,26187881,2020,80,спецалзованашкола2ступ73зпоглибленимвивченямук...,NaN


In [ ]:
datasets[2019].loc[datasets[2019]['eoname'] == 'Марянівський навчально-виховний комплекс "ДНЗ-ЗОШ" Іступеня', 'EDRPOU'] = '26224046'
datasets[2019].loc[datasets[2019]['eoname'] == 'Лобойківський заклад загальної середньої освіти Петриківської районної ради Дніпропетровської області', 'EDRPOU'] = '26460466'
datasets[2019].loc[datasets[2019]['eoname'] == 'Середня загальноосвітня школа І-ІІ ступенів села Вартиківці Кельменецького району Чернівецької області', 'EDRPOU'] = '31967606'
datasets[2019].loc[datasets[2019]['eoname'] == 'Рипенська загальноосвітня школа І ступеня Путильської районної ради Путильський район Чернівецька область', 'EDRPOU'] = '25811521'
datasets[2019].loc[datasets[2019]['eoname'] == 'Рижівська загальноосвітня школа І ступеня села Рижа Путильського району Чернівецької області', 'EDRPOU'] = '25811484'
datasets[2019].loc[datasets[2019]['eoname'] == 'Великобушинська загальноосвітня школа І-ІІ ступенів Немирівської районної ради Вінницької області', 'EDRPOU'] = '25513410'
datasets[2019].loc[datasets[2019]['eoname'] == 'Мирненська загальноосвітня школа І ступеня Надвірнянської районної ради Івано-Франківської області', 'EDRPOU'] = '20567542'
datasets[2019].loc[datasets[2019]['eoname'] == 'Завадівський навчально-виховний комплекс І-ІІ ступенів Турківської районної ради Львівської обасті', 'EDRPOU'] = '22417999'
datasets[2019].loc[datasets[2019]['eoname'] == 'Херсонський навчально-виховний комплекс "Загальноосвітня школа ІІ ступеня-ліцей журналістики, економіки та правознавства" Херсонської міської ради', 'EDRPOU'] = '24114961'

In [ ]:
datasets[2019][datasets[2019].duplicated(subset=['EDRPOU'], keep=False)]

Якщо подивитися деякі з цих ЄДРПОУ, бачимо таке:

26224046 це Марянівський навчально-виховний комплекс, а друга школа з цим єдрпоу взагалі немає єдрпоу в реєстрах

02144795 це Головне управління освіти і науки львівської обласної державної адміністрації

25811521 це Сергіївська загальноосвітня школа, а Рипенська загальноосвітня школа І ступеня це філіал, який має свій єдрпоу і т.д.

In [ ]:
datasets[2020].loc[datasets[2020]['EDRPOU'].apply(len) == 7, 'EDRPOU'] = '0' + datasets[2020].loc[datasets[2020]['EDRPOU'].apply(len) == 7, 'EDRPOU']
datasets[2020].loc[datasets[2020]['eoname'] == 'Турбівська загальноосвітня школа І-ІІІ ст. №2 _x000D_\nЛиповецької районної ради _x000D_\nВінницької області_x000D_\n_x000D_\n', 'EDRPOU'] = '21723820'
datasets[2020].loc[datasets[2020]['eoname'] == 'Середня загальноосвітня школа І - ІІІ ступенів села Уладівка Літинський район Вінницька область', 'EDRPOU'] = '26229088'
datasets[2020].loc[datasets[2020]['eoname'] == 'Середня загальноосвітня школа І-ІІІ ступенів с. Івча', 'EDRPOU'] = '26223660'
datasets[2020].loc[datasets[2020]['eoname'] == 'Комунальний заклад "Загальноосвітня школа І-ІІІ ступенів № 19 Вінницької міської ради"', 'EDRPOU'] = '26235447'
datasets[2020].loc[datasets[2020]['eoname'] == 'Тисаашванська загальноосвітня школа І-ІІ ступенів Ужгородської районної ради Закарпатської області', 'EDRPOU'] = '22103539'
datasets[2020].loc[datasets[2020]['eoname'] == 'Пшеничниківський навчально-виховний комплекс "Загальноосвітня школа І-ІІ ступенів - дошкільний навчальний заклад" Тисменицької районної ради Івано-Франківської області', 'EDRPOU'] = '20563076'
datasets[2020].loc[datasets[2020]['eoname'] == 'Старобільська загальноосвітня школа ІІ-ІІІ ступенів №4 Старобільської районної ради Луганської області', 'EDRPOU'] = '33199525'
datasets[2020].loc[datasets[2020]['eoname'] == 'Заклад загальноосвітньої середньої освіти І ступення села Смеречка Старосамбірського району Львівської області', 'EDRPOU'] = '00000000'
datasets[2020].loc[datasets[2020]['eoname'] == 'Загальноосвітня середня школа І ступеня села Дністрик Старосамбірського району Львівської області', 'EDRPOU'] = '00000000'
datasets[2020].loc[datasets[2020]['eoname'] == 'Заклад загальної середньої освіти І ступеня села Головецько-Золотнівець Старосамбірського району Львівської області', 'EDRPOU'] = '00000000'
datasets[2020].loc[datasets[2020]['eoname'] == "Воле - Блажівська середня загальноосвітня школа І ступеня Чукв'янської сільської ради Самбірського району Львівської області", 'EDRPOU'] = '22378609'
datasets[2020].loc[datasets[2020]['eoname'] == 'Загальноосвітня школа І ступеня села Виноградне', 'EDRPOU'] = '24625450'
datasets[2020].loc[datasets[2020]['eoname'] == ' Херсонський навчально-виховний комплекс "Загальноосвітня школа ІІ ст-ліцей журналістики, економіки та правознавства" Херсонської міської ради', 'EDRPOU'] = '24114961'

In [ ]:
datasets[2020][datasets[2020].duplicated(subset=['EDRPOU'], keep=False)]

In [ ]:
concatenated_df = pd.concat(list(datasets.values()), ignore_index=True)
concatenated_df

In [ ]:
concatenated_df[concatenated_df['Код ЄДРПОУ'].notna()][concatenated_df[concatenated_df['Код ЄДРПОУ'].notna()]['Код ЄДРПОУ'].apply(len) != 8]

In [ ]:
# concatenated_df[['pupilswithpreschool', 'budgetcode', 'DISO', 'Код ЄДРПОУ']] = concatenated_df[['pupilswithpreschool', 'budgetcode', 'DISO', 'Код ЄДРПОУ']].fillna(-10000)
# concatenated_df = concatenated_df.astype({'teachstuff': 'int64', 'nonteachstuff': 'int64', 'teachstuffretage': 'int64', 'pupilswithpreschool': 'int64', 'budgetcode': 'int64', 'DISO': 'int64', 'Код ЄДРПОУ': 'int64'})
# concatenated_df = concatenated_df.astype({'pupilswithpreschool': 'str', 'budgetcode': 'str', 'DISO': 'str', 'Код ЄДРПОУ': 'str'})
# concatenated_df[['pupilswithpreschool', 'budgetcode', 'DISO', 'Код ЄДРПОУ']] = concatenated_df[['pupilswithpreschool', 'budgetcode', 'DISO', 'Код ЄДРПОУ']].replace('-10000', np.NaN)
# concatenated_df

In [ ]:
concatenated_df.to_csv('Scools_stats.csv', index=False, encoding='utf-8')